# pytorch 实现 AlexNet on Fashion-MNIST

In [7]:
from __future__ import print_function

import time
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms,datasets
import matplotlib.pyplot as plt

# settings
batch_size = 64 
n_epoch = 3 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load data
transform = transforms.Compose([
    transforms.Resize(224),
#     transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=0.5, std=0.5),
])

train_dataset = datasets.FashionMNIST(root='./data', 
                                      train=True,
                                      download=True,
                                      transform=transform 
                                     )
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

test_dataset = datasets.FashionMNIST(root='./data', 
                                      train=False,
                                      download=False,
                                      transform=transform 
                                     )
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [8]:
# network
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=96,kernel_size=11,stride=4)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2)
        self.conv2 = nn.Conv2d(in_channels=96,out_channels=256,kernel_size=5,padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.dense1 = nn.Linear(256*5*5,4096)
        self.drop1 = nn.Dropout(0.5)
        self.dense2 = nn.Linear(4096,4096)
        self.drop2 = nn.Dropout(0.5)
        self.dense3 = nn.Linear(4096,10)
    
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv5(F.relu(self.conv4(F.relu(self.conv3(x)))))))
        x = x.view(-1, 256*5*5)
        x = self.dense3(self.drop2(F.relu(self.dense2(self.drop1(F.relu(self.dense1(x)))))))
        return x

In [9]:
# create model
net = AlexNet().to(device)
print(net)
critrion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

AlexNet(
  (conv1): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dense1): Linear(in_features=6400, out_features=4096, bias=True)
  (drop1): Dropout(p=0.5)
  (dense2): Linear(in_features=4096, out_features=4096, bias=True)
  (drop2): Dropout(p=0.5)
  (dense3): Linear(in_features=4096, out_features=10, bias=True)
)


In [12]:
# train
print('Training begin, use ', device)
for epoch in range(n_epoch):
    start_time = time.time()
    sum_loss = 0.0
    
    for idx, data in enumerate(train_loader, start=0):
        images, labels = data
        images, labels = Variable(images.to(device)), Variable(labels.to(device))
        
        # imshow(torchvision.utils.make_grid(images))
        # plt.show()
        # print (labels)
        optimizer.zero_grad()
        
        # print(images.shape)
        outputs = net(images)
        # print(outputs)
        loss = critrion(outputs, labels)
        loss.backward()
        optimizer.step()
        sum_loss += loss.data
        
        if idx % 99 == 0:
            end_time = time.time()
            print('[epoch {:2d}, images {:5d}] loss: {:.5f}, time: {:.3f}s'.format(
                epoch+1, (idx+1)*batch_size, sum_loss/100, (end_time-start_time)))
            start_time = time.time()
            sum_loss = 0.0 
print('Finished training!')   

# test
net.eval()
correct = 0
total = 0
for data in test_loader:
    images,labels = data
    images = images.to(device)
    labels = labels.to(device)
    outputs = net(Variable(images))
    _,pred = torch.max(outputs,1)
    total += labels.size(0)
    correct += (pred == labels).sum()
print('Accuracy of the network on the {} test images: {:.4f}%'.format(total , 100.0 * correct / total))

Training begin, use  cuda


IndexError: Traceback (most recent call last):
  File "/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torchvision/datasets/mnist.py", line 95, in __getitem__
    img = self.transform(img)
  File "/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 60, in __call__
    img = t(img)
  File "/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 163, in __call__
    return F.normalize(tensor, self.mean, self.std, self.inplace)
  File "/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-